<a href="https://colab.research.google.com/github/oveay/Contur_TEST/blob/main/RuBERT_end.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Обучение модели на основе эмбеддингов

Необходимо преобразовать текстовые данные, чтобы можно было подать их на вход модели. Можно разными способами закодировать слова, либо символы, создать словарь токенов и заменить каждый токен на его номер. Но в таком случае наша модель не будет иметь информации о самих словах и что они значат в языке.

Первым решением данной проблемы могут стать предобученные эмбеддинги для слов (например fasttext), тогда модель будет иметь представление о каждом отдельно взятом токене, но не будет знать ничего о слове в конкретном контексте входных данных. Можно было бы сверху на этих эмбеддингах обучить, например biderectional LSTM и таким образом получить представление предложений, но в данном блокноте рассматривается другой подход.

С помощью предобученной модели - RuBERT, будут получены векторные представления предложений, которые уже хранят информацию о нем, а далее, поверх новых эмбеддингов, будут обучены модели классификации на исходную задачу.

In [ ]:
pip install transformers

In [ ]:
import torch
import pandas as pd
import numpy as np

# Библиотека с предобученными трансформерами
from transformers import AutoTokenizer, AutoModel

# Для разбиения train выборки
from sklearn.model_selection import train_test_split

# Библиотеки для классификации
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
import xgboost as xgb

# Для подсчета метрик
from sklearn.metrics import f1_score, accuracy_score, roc_auc_score

In [ ]:
# Константы

RANDOM_SEED = 42
TRAIN_FILENAME = 'train.tsv'
TEST_FILENAME = 'test.tsv'
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

### Считывание данных

In [ ]:
def read_tsv(filename):
  df = pd.read_csv(filename, sep="\t")
  return (df[i] for i in df) # Возвращаем итератор столбцов датафрейма

In [ ]:
X, y = read_tsv(TRAIN_FILENAME) # Распаковываем итератор
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=RANDOM_SEED)

### Токенизация данных

In [ ]:
def get_tokens(data):
  tokenized = tokenizer(list(data), padding=True, truncation=True, max_length=24, return_tensors='pt')
  tokenized = tokenized.to(DEVICE)
  return tokenized

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("sberbank-ai/sbert_large_nlu_ru")

In [ ]:
tokenized_train = get_tokens(X_train)
tokenized_val = get_tokens(X_val)

### Получение эмбеддингов предложений

In [ ]:
# Скачивание предобученной модели
model_RuBERT = AutoModel.from_pretrained("sberbank-ai/sbert_large_nlu_ru").to(DEVICE)

In [ ]:
''' Данная функция взята с https://huggingface.co '''


#Mean Pooling - Take attention mask into account for correct averaging
def mean_pooling(model_output, attention_mask):
    token_embeddings = model_output[0] #First element of model_output contains all token embeddings
    input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
    sum_embeddings = torch.sum(token_embeddings * input_mask_expanded, 1)
    sum_mask = torch.clamp(input_mask_expanded.sum(1), min=1e-9)
    return sum_embeddings / sum_mask

In [ ]:
def get_sentence_embeddings(tokenized):
  with torch.no_grad():
    model_output = model_RuBERT(**tokenized)
  
  return mean_pooling(model_output, tokenized['attention_mask'])


train_embeddings = get_sentence_embeddings(tokenized_train)
val_embeddings = get_sentence_embeddings(tokenized_val)

### Построение моделей классификации

In [ ]:
classification_models = dict()

clf = make_pipeline(StandardScaler(), SVC(kernel='rbf', gamma='scale'))
clf.fit(train_embeddings.cpu(), y_train)
classification_models['SVM'] = clf

clf = make_pipeline(StandardScaler(), RandomForestClassifier())
clf.fit(train_embeddings.cpu(), y_train)
classification_models['RandomForestClassifier'] = clf

clf = make_pipeline(StandardScaler(), LogisticRegression())
clf.fit(train_embeddings.cpu(), y_train)
classification_models['LogisticRegeression'] = clf

clf = make_pipeline(StandardScaler(), xgb.XGBClassifier())
clf.fit(train_embeddings.cpu(), y_train)
classification_models['GBM'] = clf

### Подсчет метрик

In [ ]:
def compute_metrics(y_true, y_pred):
  metric_scores = dict()

  metric_scores['f1_score'] = f1_score(y_true, y_pred)
  metric_scores['roc-auc'] = roc_auc_score(y_true, y_pred)
  metric_scores['accuracy'] = accuracy_score(y_true, y_pred)

  return metric_scores

In [ ]:
for model_name, class_model in classification_models.items():
  y_pred = np.array(class_model.predict(val_embeddings.cpu()))
  metrics = compute_metrics(y_val, y_pred)

  print(f'{model_name}:')
  for score_name, score in metrics.items():
    print(f'{score_name}: {score}')
  print()

SVM:
f1_score: 0.9084687767322498
roc-auc: 0.9072953736654805
accuracy: 0.9071180555555556

RandomForestClassifier:
f1_score: 0.8568980291345331
roc-auc: 0.8552234754810302
accuracy: 0.8550347222222222

LogisticRegeression:
f1_score: 0.874251497005988
roc-auc: 0.8725526268170577
accuracy: 0.8723958333333334

GBM:
f1_score: 0.8688245315161841
roc-auc: 0.866367090898124
accuracy: 0.8663194444444444



### Анализ

Наилучший результат показывает SVM, в районе 0.9 - 0.91 по каждой из трех метрик, поэтому именно этот тип модели классификации мы будем использовать для разметки тестовых данных.

### Обучение полной модели

В предыдущих разделах мы разделяли данную нам тренировочную выборку на непосредственно train и validation, чтобы выбрать модель классификации и получить значения метрик.

Но основная задача стоит в разметке данных из файла 'test.tsv', поэтому нет смысла обучать модель лишь на часте тренировочных данных

In [ ]:
# X, y - полные данные, которые мы сплитовали

tokenized_data = get_tokens(X)
data_embeddings = get_sentence_embeddings(tokenized_data)

classificator = make_pipeline(StandardScaler(), SVC(kernel='rbf', gamma='scale'))
classificator.fit(data_embeddings.cpu(), y)

In [ ]:
X_test, _ = read_tsv(TEST_FILENAME) # y - не нужен, так как это неразмеченные данные

tokenized_test = get_tokens(X_test)
test_embeddings = get_sentence_embeddings(tokenized_test)

y_predictions = classificator.predict(test_embeddings.cpu())

### Запись данных в файл

In [ ]:
predictions = pd.DataFrame({'title': X_test, 'is_fake': y_predictions})
predictions.set_index('title', inplace=True)

In [ ]:
predictions.to_csv('predictions.tsv', sep="\t")